<a href="https://colab.research.google.com/github/VinicyusSantos/Teste_Analytics_VinicyusSantos/blob/main/notebooks/Limpeza_e_Analise_de_Dados_de_Vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simulação dos Dados**

In [1]:
#importando bibliotecas
import pandas as pd
import numpy as np
import random

In [2]:
# Garantir reprodutibilidade
random.seed(42)
np.random.seed(42)

#criação do periodo descrito
datas = pd.date_range(start="2023-01-01", end="2023-12-31")

# Produtos, categorias e preços fixos
produtos_info = {
    "Notebook": {"Categoria": "Eletrônicos", "Preço": 3500.00},
    "Smartphone": {"Categoria": "Eletrônicos", "Preço": 2000.00},
    "Geladeira": {"Categoria": "Eletrodomésticos", "Preço": 2500.00},
    "Fogão": {"Categoria": "Eletrodomésticos", "Preço": 1500.00},
    "Camiseta": {"Categoria": "Vestuário", "Preço": 80.00},
    "Calça": {"Categoria": "Vestuário", "Preço": 120.00},
    "Tênis": {"Categoria": "Calçados", "Preço": 300.00},
    "Sandália": {"Categoria": "Calçados", "Preço": 150.00},
    "Livro": {"Categoria": "Papelaria", "Preço": 50.00},
    "Caneta": {"Categoria": "Papelaria", "Preço": 5.00},
}

lista_produtos = list(produtos_info.keys())

In [3]:
# Função para simular sazonalizade, tendo em vista que alguns produtos possuem picos de compras em certos periodos
#Também com o intuito de deixar os dados mais proximos da realidade
def escolher_produto(data):
    mes = data.month
    if mes in [1, 2]:  # Verão / roupas
        pesos = [1, 1, 1, 1, 5, 5, 3, 3, 1, 1]
    elif mes in [3, 4, 5]:  # Volta às aulas
        pesos = [1, 1, 1, 1, 2, 2, 1, 1, 6, 6]
    elif mes in [6, 7]:  # Férias / eletrônicos
        pesos = [5, 5, 2, 2, 1, 1, 1, 1, 1, 1]
    elif mes in [8, 9, 10]:  # Renovação casa
        pesos = [2, 2, 5, 5, 1, 1, 1, 1, 1, 1]
    elif mes in [11, 12]:  # Black Friday e Natal
        pesos = [6, 6, 3, 3, 5, 5, 4, 4, 2, 2]
    else:
        pesos = [1] * len(lista_produtos)
    return random.choices(lista_produtos, weights=pesos, k=1)[0]

In [4]:
# Criar vendas diárias (1 a 5 vendas por dia). garantindo que sempre haverá vendas
registros = []
id_counter = 1

for data in datas:
    numero_vendas = random.randint(1, 5)
    for _ in range(numero_vendas):
        produto = escolher_produto(data)
        categoria = produtos_info[produto]["Categoria"]
        preco = produtos_info[produto]["Preço"]
        registros.append([
            id_counter,
            data,
            produto,
            categoria,
            np.random.randint(1, 10),  # Quantidade
            preco
        ])
        id_counter += 1

#criando o dataframe
df = pd.DataFrame(registros, columns=["ID", "Data", "Produto", "Categoria", "Quantidade", "Preço"])

In [5]:
# Inserindo 5 valores faltantes aleatórios
for _ in range(5):
    linha = random.randint(0, len(df)-1)
    coluna = random.choice(df.columns[1:])  # sem NaN no id
    df.at[linha, coluna] = np.nan

# Adicionar 2 registros duplicados
duplicatas = df.sample(2, random_state=42)
df = pd.concat([df, duplicatas], ignore_index=True)

In [6]:
print(df.head(10))
print("\nTotal de registros:", len(df))
print("\nValores faltantes por coluna:\n", df.isna().sum())
print("\nDuplicados no dataset:",df.duplicated().sum())

   ID       Data     Produto         Categoria  Quantidade   Preço
0   1 2023-01-01    Notebook       Eletrônicos         7.0  3500.0
1   2 2023-01-02    Camiseta         Vestuário         4.0    80.0
2   3 2023-01-02       Fogão  Eletrodomésticos         8.0  1500.0
3   4 2023-01-02   Geladeira  Eletrodomésticos         5.0  2500.0
4   5 2023-01-03  Smartphone       Eletrônicos         7.0  2000.0
5   6 2023-01-03       Calça         Vestuário         3.0   120.0
6   7 2023-01-03    Notebook       Eletrônicos         7.0  3500.0
7   8 2023-01-03    Camiseta         Vestuário         8.0    80.0
8   9 2023-01-03       Calça         Vestuário         5.0   120.0
9  10 2023-01-04       Calça         Vestuário         4.0   120.0

Total de registros: 1080

Valores faltantes por coluna:
 ID            0
Data          0
Produto       1
Categoria     2
Quantidade    1
Preço         1
dtype: int64

Duplicados no dataset: 2


# **Limpeza dos Dados**

In [7]:
df.head()

,ID,Data,Produto,Categoria,Quantidade,Preço
0,1,2023-01-01,Notebook,Eletrônicos,7.0,3500.0
1,2,2023-01-02,Camiseta,Vestuário,4.0,80.0
2,3,2023-01-02,Fogão,Eletrodomésticos,8.0,1500.0
3,4,2023-01-02,Geladeira,Eletrodomésticos,5.0,2500.0
4,5,2023-01-03,Smartphone,Eletrônicos,7.0,2000.0


**Nulos**

In [8]:
#verificando a quantidade de nulos
df.isnull().sum()

,0
ID,0
Data,0
Produto,1
Categoria,2
Quantidade,1
Preço,1


In [9]:
#localizando onde estão os dados nulos com .loc
df.loc[df['Produto'].isnull() | df['Categoria'].isnull() |df['Quantidade'].isnull() | df['Preço'].isnull()]

,ID,Data,Produto,Categoria,Quantidade,Preço
187,188,2023-03-03,NaN,Papelaria,4.0,50.0
301,302,2023-04-11,Calça,Vestuário,NaN,120.0
328,329,2023-04-17,Calça,NaN,4.0,120.0
920,921,2023-11-04,Geladeira,Eletrodomésticos,8.0,NaN
923,924,2023-11-06,Calça,NaN,5.0,120.0


In [10]:
#função para verificar os nulos com base no produto, categoria ou preço.
#ex: se o produto for nulo, verifica a categoria e o preço para classifica-lo
def verificador_de_nulos(produto, categoria, preco):
    if pd.isnull(produto):
        if (categoria == 'Eletrônicos') and (preco >= 3500):
            return 'Notebook'
        elif (categoria == 'Eletrônicos') and (preco < 3500):
            return 'Smartphone'
        elif (categoria == 'Eletrodomésticos') and (preco >= 2000):
            return 'Geladeira'
        elif (categoria == 'Eletrodomésticos') and (preco < 2000):
            return 'Fogão'
        elif (categoria == 'Vestuário') and (preco <= 100):
            return 'Camiseta'
        elif (categoria == 'Vestuário') and (preco > 100):
            return 'Calça'
        elif (categoria == 'Calçados') and (preco >= 200):
            return 'Tênis'
        elif (categoria == 'Calçados') and (preco < 200):
            return 'Sandália'
        elif (categoria == 'Papelaria') and (preco > 10):
            return 'Livro'
        elif (categoria == 'Papelaria') and (preco <= 10):
            return 'Caneta'
        return None

    elif pd.isnull(categoria):
        if produto == 'Notebook' or produto == 'Smartphone':
            return 'Eletrônicos'
        elif produto == 'Geladeira' or produto == 'Fogão':
            return 'Eletrodomésticos'
        elif produto == 'Camiseta' or produto == 'Calça':
            return 'Vestuário'
        elif produto == 'Tênis' or produto == 'Sandália':
            return 'Calçados'
        elif produto == 'Livro' or produto == 'Caneta':
            return 'Papelaria'
        return None

    elif pd.isnull(preco):
        precos_fixos = {
            "Notebook": 3500.00,
            "Smartphone": 2000.00,
            "Geladeira": 2500.00,
            "Fogão": 1500.00,
            "Camiseta": 80.00,
            "Calça": 120.00,
            "Tênis": 300.00,
            "Sandália": 150.00,
            "Livro": 50.00,
            "Caneta": 5.00
        }
        return precos_fixos.get(produto, None)

    return None

In [11]:
#aplicando o verificador de nulos
df['Produto'] = df.apply(
    lambda row: verificador_de_nulos(row['Produto'], row['Categoria'], row['Preço'])
    if pd.isnull(row['Produto']) else row['Produto'], axis=1
)
df['Categoria'] = df.apply(
    lambda row: verificador_de_nulos(row['Produto'], row['Categoria'], row['Preço'])
    if pd.isnull(row['Categoria']) else row['Categoria'], axis=1
)
df['Preço'] = df.apply(
    lambda row: verificador_de_nulos(row['Produto'], row['Categoria'], row['Preço'])
    if pd.isnull(row['Preço']) else row['Preço'], axis=1
)
df['Quantidade'] = df['Quantidade'].fillna(df['Quantidade'].median())

df.isnull().sum()

,0
ID,0
Data,0
Produto,0
Categoria,0
Quantidade,0
Preço,0


**Duplicados**

In [12]:
#verificação de duplicados
df.duplicated().sum()

np.int64(2)

In [13]:
df.loc[df.duplicated()]

,ID,Data,Produto,Categoria,Quantidade,Preço
1078,424,2023-05-18,Livro,Papelaria,4.0,50.0
1079,549,2023-07-01,Geladeira,Eletrodomésticos,5.0,2500.0


In [14]:
print(df.loc[df['ID'] == 549])
print(df.loc[df['ID'] == 424])

       ID       Data    Produto         Categoria  Quantidade   Preço
548   549 2023-07-01  Geladeira  Eletrodomésticos         5.0  2500.0
1079  549 2023-07-01  Geladeira  Eletrodomésticos         5.0  2500.0
       ID       Data Produto  Categoria  Quantidade  Preço
423   424 2023-05-18   Livro  Papelaria         4.0   50.0
1078  424 2023-05-18   Livro  Papelaria         4.0   50.0


In [15]:
df.drop_duplicates(keep="first",inplace=True)

In [16]:
df.duplicated().sum()

np.int64(0)

**Salvando os dados**

In [17]:
import os

if not os.path.exists("data/data_clean.csv"):
    df.to_csv("data/data_clean.csv", index=False)
else:
    df = pd.read_csv("data/data_clean.csv")

# **Análise dos Dados**

In [18]:
#lendo os dados limpos
df = pd.read_csv("data/data_clean.csv")
df.head()

,ID,Data,Produto,Categoria,Quantidade,Preço
0,1,2023-01-01,Notebook,Eletrônicos,7.0,3500.0
1,2,2023-01-02,Camiseta,Vestuário,4.0,80.0
2,3,2023-01-02,Fogão,Eletrodomésticos,8.0,1500.0
3,4,2023-01-02,Geladeira,Eletrodomésticos,5.0,2500.0
4,5,2023-01-03,Smartphone,Eletrônicos,7.0,2000.0


In [19]:
#Calcule o total de vendas (Quantidade * Preço) por produto.

#adicionando coluna de total de vendas diarias
df['Total_Vendas'] = df['Quantidade'] * df['Preço']

#agora calculando o total
agrupamento = df.groupby('Produto')['Total_Vendas'].sum()
print(agrupamento)

Produto
Calça           59280.0
Camiseta        46160.0
Caneta           3130.0
Fogão          838500.0
Geladeira     1447500.0
Livro           27200.0
Notebook      1886500.0
Sandália        57900.0
Smartphone    1282000.0
Tênis          100500.0
Name: Total_Vendas, dtype: float64


In [20]:
#Identifique o produto com o maior número de vendas totais.
maior_nvendas = agrupamento.idxmax()
print(maior_nvendas)

Notebook
